In [ ]:
from RawHandler.RawHandler import RawHandler
from RawHandler.utils import download_file_requests
import matplotlib.pyplot as plt
import os

In [ ]:
# Download example raw file.
file_url = "https://dataverse.uclouvain.be/api/access/datafile/:persistentId?persistentId=doi:10.14428/DVN/DEQCIM/LWSLMG"

output_file = "example_raw.arw"

if not os.path.exists(output_file):
    download_file_requests(file_url, output_file)

In [ ]:
rh = RawHandler("example_raw.arw")

In [ ]:
# Shows the raw pattern

rh.bayer_pattern_description()

In [ ]:
# Converts raw data to RGB data, and crop
# The image needs to be transposed to put the channels last as it is by default set up to return channles first as is torch's standard.
# The output is quite green as we haven't applied a colorspace yet.
img = rh.as_rgb()
plt.imshow(img.transpose(1, 2, 0))

In [ ]:
# Now, the colors look nicer, but still quite dark as it is in a linear color space.
img = rh.as_RGB_colorspace(img=img, colorspace="AdobeRGB")
plt.imshow(img.transpose(1, 2, 0))